In [9]:
import json
import math
import glob

def mean(values):
    return sum(values) / len(values) if values else 0.0

def std_dev(values):
    filtered = [v for v in values if v != 0]
    if len(filtered) <= 1:
        return 0.0
    m = mean(filtered)
    variance = sum((x - m) ** 2 for x in filtered) / len(filtered)
    return math.sqrt(variance)

def analyze_json_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)

    sprint_risks = []
    sprint_uncertainties = []
    sprint_capacity_usage = []

    for solution in data.get("solutions", []):
        for sprint in solution.get("sprints", []):
            stories = sprint.get("stories", [])

            if not stories:
                continue  # considera solo sprint occupati

            total_risk = sum(story.get("risk", 0) for story in stories)
            total_uncertainty = sum(story.get("uncertainty", 0) for story in stories)

            capacity = sprint.get("capacity", 0)
            used_capacity = sprint.get("used_capacity", 0)

            if capacity > 0:
                sprint_capacity_usage.append(used_capacity / capacity)

            sprint_risks.append(total_risk)
            sprint_uncertainties.append(total_uncertainty)

    return {
        "file": filename,
        "num_sprint_considered": len(sprint_risks),

        # Rischio
        "risk_std_dev": std_dev(sprint_risks),
        "risk_max": max(sprint_risks) if sprint_risks else 0.0,
        "risk_min": min(sprint_risks) if sprint_risks else 0.0,

        # Incertezza
        "uncertainty_std_dev": std_dev(sprint_uncertainties),
        "uncertainty_max": max(sprint_uncertainties) if sprint_uncertainties else 0.0,
        "uncertainty_min": min(sprint_uncertainties) if sprint_uncertainties else 0.0,

        # Capacità
        "avg_capacity_usage": mean(sprint_capacity_usage)
    }


In [ ]:

json_files = glob.glob("*.json")
output_file = "resoconto.txt"

print("=== RESOCONTO PER FILE ===\n")

with open(output_file, "w", encoding="utf-8") as f_out:
    f_out.write("=== RESOCONTO PER FILE ===\n\n")

for file in json_files:
    r = analyze_json_file(file)

    print(
        f"File: {r['file']} | "
        f"Sprint: {r['num_sprint_considered']} | "
        f"STD Rischio: {r['risk_std_dev']:.3f} | "
        f"Rischio max/min: {r['risk_max']:.3f}/{r['risk_min']:.3f} | "
        f"STD Incertezza: {r['uncertainty_std_dev']:.3f} | "
        f"Incertezza max/min: {r['uncertainty_max']:.3f}/{r['uncertainty_min']:.3f} | "
        f"Utilizzo medio capacità: {r['avg_capacity_usage']:.2%}"
    )

    line = (
            f"{r['file']} &  & "
            f"{r['num_sprint_considered']} & "
            f"{r['risk_std_dev']:.3f} & "
            f"{r['risk_max']:.3f}  &  {r['risk_min']:.3f} & "
            f"{r['uncertainty_std_dev']:.3f} & "
            f"{r['uncertainty_max']:.3f}  &  {r['uncertainty_min']:.3f} & "
            f"{r['avg_capacity_usage']:.2%} \\\\\n"
        )

=== RESOCONTO PER FILE ===

File: solutionCLEX1.json | Sprint: 5 | STD Rischio: 2.227 | Rischio max/min: 10.600/5.100 | STD Incertezza: 2.133 | Incertezza max/min: 10.100/4.300 | Utilizzo medio capacità: 95.59%
